In [1]:
import asyncio
import awswrangler as wr
from datetime import datetime, timedelta
from metaapi_cloud_sdk import MetaApi
import os
import pytz
import sys

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/workflow") 

In [3]:
from src.config import *
from src.io import *
from src.utils import *
from src.access import get_demo_account

In [4]:
async def yield_raw_ticks_between_datetimes(
    account: object,
    symbol: str,
    datetimeutc_from: datetime,
    datetimeutc_to: datetime,
    pause: int = 5
):
    offset = 0
    while True:
        l_ticks_i = await account.get_historical_ticks(
            symbol,
            start_time=datetimeutc_from,
            limit=1000,
            offset=offset,
        )
        if len(l_ticks_i) == 0:
            break
        yield offset, l_ticks_i
        await asyncio.sleep(pause)
        datetimeutc_last = l_ticks_i[-1]["time"]
        offset += len(l_ticks_i)
        if datetimeutc_last >= datetimeutc_to:
            break

In [5]:
symbol = "GBPUSD"

In [6]:
datetimestr_from = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now())
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d000000000000")
)

datetimestr_to = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now() + timedelta(days=1))
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d%H%M%S%f")
)
datetimestr_from, datetimestr_to

('20220701000000000000', '20220702122007690707')

In [7]:
# Get demo account
account = await get_demo_account()

# Convert datetimes from string to datetime
datetimeutc_from = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_from, "%Y%m%d%H%M%S%f"))
datetimeutc_to = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_to, "%Y%m%d%H%M%S%f"))

In [8]:
generator_async = yield_raw_ticks_between_datetimes(
    account=account,
    symbol=symbol,
    datetimeutc_from=datetimeutc_from,
    datetimeutc_to=datetimeutc_to,
)

In [11]:
# async for offset, ticks in generator_async:
#     print(offset)

In [12]:
ticks

[{'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 1, 0, 1, 23, 541000, tzinfo=datetime.timezone.utc),
  'last': 1.21566,
  'volume': 150000,
  'brokerTime': '2022-07-01 03:01:23.541'},
 {'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 1, 0, 1, 23, 875000, tzinfo=datetime.timezone.utc),
  'last': 1.21566,
  'volume': 150000,
  'brokerTime': '2022-07-01 03:01:23.875'},
 {'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 1, 0, 1, 23, 957000, tzinfo=datetime.timezone.utc),
  'last': 1.21566,
  'volume': 150000,
  'brokerTime': '2022-07-01 03:01:23.957'},
 {'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 1, 0, 1, 24, 8000, tzinfo=datetime.timezone.utc),
  'last': 1.21566,
  'volume': 150000,
  'brokerTime': '2022-07-01 03:01:24.008'},
 {'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 1, 0, 1, 24, 58000, tzinfo=datetime.timezone.utc),
  'last': 1.21566,
  'volume': 150000,
  'brokerTime': '2022-07-01 03:01:24.058'},
 {'symbol': 'GBPUSD',
  'time': d